# Read filtered alignments and find and count corresponding barcodes

We include only insert-barcode pairs that are supported 10 or more reads.

In [1]:
import pandas as pd
import numpy as np
import gzip
import sys
from Bio.SeqIO.QualityIO import FastqGeneralIterator

## Define analysis-specific parameters

In [7]:
fastq_file = sys.argv[1]
alignment_file = sys.argv[2]
output_file = sys.argv[3]
#fastq_file = '../data/fastq/100p10.R1.fastq.gz'
#alignment_file = '../data/filtered_alignments/didi_2_grna.tsv.gz'
#output_file = '../data/insert_barcode_counts/didi_2_grna.tsv.gz'

## Read in the filtered alignments table

In [8]:
aln = pd.read_table(alignment_file).sort_values(by = 'qname').set_index('qname')

FileNotFoundError: [Errno 2] No such file or directory: '../data/filtered_alignments/didi_2_grna.tsv.gz'

## Parse barcode from original FASTQ read for each filtered alignment

In [9]:
fastq_records = FastqGeneralIterator(gzip.open(fastq_file, 'rt'))
read = next(fastq_records)
barcodes =  np.chararray(len(aln), itemsize='24', unicode=True) 
for pos, row in enumerate(aln.iterrows()):
    while int(read[0]) != row[0]:
        read = next(fastq_records)
    barcodes[pos] = read[1][3:27]

OSError: Not a gzipped file (b'NC')

## Count the number of reads for each barcode-insert pair and filter to n_reads >= 10

In [6]:
aln['barcode'] = barcodes

NameError: name 'barcodes' is not defined

In [91]:
barcode_insert_counts = (aln
                         .groupby(['barcode', 'ref'])
                         .size()
                         .to_frame(name='read_count')
                         .reset_index()
                         .rename(columns={'ref': 'insert_num'})
                         .sort_values(by='read_count', ascending=False)
                         )
barcode_insert_counts = barcode_insert_counts.loc[barcode_insert_counts['read_count'] >= 10]
barcode_insert_counts.to_csv(output_file, sep = '\t', index = False)